In [ ]:
import sionna as sn
import numpy as np
import tensorflow as tf

In [ ]:
BATCH_SIZE = 300

In [ ]:
class CellModel(tf.keras.Model):
    
    def __init__(self, scenario='umi', los=None, ground_users=False, num_bs_ant=8):

        super().__init__()

        self.NUM_BS_ANT = num_bs_ant
        self.NUM_UT = 8
        self.NUM_STREAMS_PER_TX = 2

        self.NUM_BITS_PER_SYMBOL = 4
        self.CODERATE = 0.5

        self.BS_HEIGHT = 15.0


        self.CARRIER_FREQUENCY = 1.9e9

        if ground_users:
            self.MAX_UT_HEIGHT = 2.0
            self.MIN_UT_HEIGHT = 1.0
        else:
            self.MAX_UT_HEIGHT = 300
            self.MIN_UT_HEIGHT = 20.0

        self.rx_tx_association = np.ones([1, self.NUM_UT])

        self.stream_management = sn.mimo.StreamManagement(
            rx_tx_association=self.rx_tx_association,
            num_streams_per_tx=self.NUM_STREAMS_PER_TX 
        )

        self.resource_grid = sn.ofdm.ResourceGrid(
            num_ofdm_symbols=11,
            fft_size=16,
            subcarrier_spacing=5e3,
            num_tx=self.NUM_UT,
            num_streams_per_tx=self.NUM_STREAMS_PER_TX,
            cyclic_prefix_length=6,
            pilot_pattern="kronecker",
            pilot_ofdm_symbol_indices=[1,9]
        )

        self.resource_grid_mapper = sn.ofdm.ResourceGridMapper(resource_grid=self.resource_grid)
        self.frequencies = sn.channel.subcarrier_frequencies(
            num_subcarriers=self.resource_grid.fft_size,
            subcarrier_spacing=self.resource_grid.subcarrier_spacing
        )

        NUM_CODED_BITS = int(self.resource_grid.num_data_symbols * self.NUM_BITS_PER_SYMBOL)
        self.NUM_INFO_BITS = int(NUM_CODED_BITS * self.CODERATE)

        self.binary_source = sn.utils.BinarySource()

        constellation = sn.mapping.Constellation('qam', self.NUM_BITS_PER_SYMBOL)
        self.mapper = sn.mapping.Mapper(constellation=constellation)
        self.demapper = sn.mapping.Demapper('app', constellation=constellation)

        self.encoder = sn.fec.ldpc.LDPC5GEncoder(self.NUM_INFO_BITS, NUM_CODED_BITS)
        self.decoder = sn.fec.ldpc.LDPC5GDecoder(encoder=self.encoder, hard_out=True)

        self.ls_est = sn.ofdm.LSChannelEstimator(resource_grid=self.resource_grid, interpolation_type='nn')

        self.lmmse_equ = sn.ofdm.LMMSEEqualizer(resource_grid=self.resource_grid, stream_management=self.stream_management)

        self.channel_freq = sn.channel.ApplyOFDMChannel(add_awgn=True)

        self.ut_array = sn.channel.tr38901.Antenna(
            polarization='dual',
            polarization_type='cross',
            antenna_pattern='omni',
            carrier_frequency=self.CARRIER_FREQUENCY
        )

        self.bs_array = sn.channel.tr38901.AntennaArray(
            num_rows=1,
            num_cols=self.NUM_BS_ANT,
            polarization='dual',
            polarization_type='cross',
            antenna_pattern='38.901',
            carrier_frequency=self.CARRIER_FREQUENCY
        )

        topology = sn.channel.gen_single_sector_topology(
            batch_size=BATCH_SIZE,
            indoor_probability=0,
            num_ut=self.NUM_UT,
            scenario=scenario,
            bs_height=self.BS_HEIGHT,
            max_ut_height=self.MAX_UT_HEIGHT,
            min_ut_height=self.MIN_UT_HEIGHT
        )

        if scenario == 'umi':
            self.channel_model = sn.channel.tr38901.UMi(
                carrier_frequency=self.CARRIER_FREQUENCY,
                o2i_model='low',
                ut_array=self.ut_array,
                bs_array=self.bs_array,
                direction='uplink',
                enable_pathloss=True,
                enable_shadow_fading=True
            )
        elif scenario == 'uma':
            self.channel_model = sn.channel.tr38901.UMa(
                carrier_frequency=self.CARRIER_FREQUENCY,
                o2i_model="low",
                ut_array=self.ut_array,
                bs_array=self.bs_array,
                direction='uplink',
                enable_pathloss=True,
                enable_shadow_fading=True
            )
    
        self.channel_model.set_topology(*topology, los=los)

    @tf.function
    def __call__(self, batch_size, ebno_db):

        no = sn.utils.ebnodb2no(
            coderate=self.CODERATE,
            num_bits_per_symbol=self.NUM_BITS_PER_SYMBOL,
            resource_grid=self.resource_grid,
            ebno_db=ebno_db
        ) 

        bits = self.binary_source([
            batch_size,
            self.NUM_UT,
            self.resource_grid.num_streams_per_tx,
            self.NUM_INFO_BITS
        ])
        coded_bits = self.encoder(bits)
        qam_symbols = self.mapper(coded_bits)
        ofdm_symbols = self.resource_grid_mapper(qam_symbols)

        a, tau = self.channel_model(
            num_time_samples=self.resource_grid.num_ofdm_symbols,
            sampling_frequency=1/self.resource_grid.ofdm_symbol_duration
        )
        h_freq = sn.channel.cir_to_ofdm_channel(
            frequencies=self.frequencies,
            a=a,
            tau=tau,
            normalize=True
        )
        received_symbols = self.channel_freq([ofdm_symbols, h_freq, no])
        estimation, err_var = self.ls_est([received_symbols, no])
        equalized_symbols, no_eff = self.lmmse_equ([ received_symbols, estimation, err_var, no])
        llr = self.demapper([equalized_symbols, no_eff])
        bits_hat = self.decoder(llr)

        return bits, bits_hat
                

In [ ]:
ber_plots = sn.utils.PlotBER('UAV Simulation')

umi_uav = CellModel('umi', los=True)
uma_uav = CellModel('uma', los=True)
umi_ground_los = CellModel('umi', ground_users=True, los=True)
uma_ground_los = CellModel('uma', ground_users=True, los=True)
umi_ground_nlos = CellModel('umi', ground_users=True, los=False)
uma_ground_nlos = CellModel('uma', ground_users=True, los=False)

In [ ]:
umi_uav.channel_model.show_topology();
umi_ground_nlos.channel_model.show_topology();

In [ ]:
ber_plots.simulate(
  umi_uav,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMi',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_uav,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMa',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  umi_ground_los,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMi e LOS',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_ground_los,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMa e LOS',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  umi_ground_nlos,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMi e NLOS',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_ground_nlos,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMa e NLOS',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots();

In [ ]:
ber_plots = sn.utils.PlotBER('Simulação comparação UAV 8 antenas e 16 antenas')

umi_uav_16ant = CellModel('umi', los=True, num_bs_ant=16)
uma_uav_16ant = CellModel('uma', los=True, num_bs_ant=16)
umi_uav_8ant = CellModel('umi', los=True)
uma_uav_8ant = CellModel('uma', los=True)

In [ ]:
ber_plots.simulate(
  umi_uav_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMi e 16 antenas',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_uav_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMa e 16 antenas',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  umi_uav_8ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMi e 8 antenas',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_uav_8ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMa e 8 antenas',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots();

In [ ]:
ber_plots = sn.utils.PlotBER('Simulação UAV com 16 antenas')

umi_uav_16ant = CellModel('umi', los=True, num_bs_ant=16)
uma_uav_16ant = CellModel('uma', los=True, num_bs_ant=16)
umi_ground_16ant = CellModel('umi', los=False,  ground_users=True, num_bs_ant=16)
uma_ground_16ant = CellModel('uma', los=False,  ground_users=True, num_bs_ant=16)

In [ ]:
ber_plots.simulate(
  umi_uav_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMi',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_uav_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='UAV com UMa',
  soft_estimates=False,
  max_mc_iter=1000,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  umi_ground_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMi',
  soft_estimates=False,
  max_mc_iter=100,
  show_fig=False
)

In [ ]:
ber_plots.simulate(
  uma_ground_16ant,
  batch_size=BATCH_SIZE,
  ebno_dbs=np.linspace(-5,10,16),
  num_target_bit_errors=1000,
  legend='Solo com UMa',
  soft_estimates=False,
  max_mc_iter=100,
  show_fig=False
)

In [ ]:
ber_plots()